En esta sección vamos a hacer un ejemplo completo de como hacer un Análisis Exploratorio de Datos (EDA en inglés).

Hoy en dia no hay un proceso standard a la hora de realizar EDA, pero un proceso que a mi me gusta está basado en el proceso propuesto por Distric Data Labs ([aquí](http://blog.districtdatalabs.com/data-exploration-with-python-1) un blog post, y [aquí](https://www.youtube.com/watch?v=YEBRkLo568Q) una charla sobre el proceso).

In [1]:
%load_ext watermark
%watermark

Last updated: 2023-06-05T11:56:13.962164-05:00

Python implementation: CPython
Python version       : 3.10.9
IPython version      : 8.10.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 8
Architecture: 64bit



El archivo original está en: https://www.fueleconomy.gov/feg/epadata/vehicles.csv.zip
El archivo que vamos a usar es una versión modificada (con menos columnas)

Descripcion del dataset
http://www.fueleconomy.gov/feg/ws/index.shtml#ft7


Supongamos que somos la agencia de porteccion ambiental americana, la EPA. Uno de sus trabajos es analizar los coches nuevos que se venden en EEUU y estudiar su contaminacion.

In [2]:
import pandas as pd

In [3]:
%matplotlib notebook

### Lectura de datos

In [4]:
df_consumo_trabajo = pd.read_csv("Data/dataset-final-trabajo.csv")

In [5]:
df_consumo_trabajo.shape

(28691, 59)

In [6]:
df_consumo_trabajo.head()

,directorio,per_sexo,per_edad,vive_padre_hogar,vive_madre_hogar,departamento,municipio,tipo_vivienda,servicio_energia,estrato,...,r_01_accidente_laboral,r_02_consumio_sp_en_accidente_laboral,r_03_estuvo_enfermo_accidentado_ult_30d,r_04_consecuencia_sp,r_05_existe_reglamento_sp_trabajo,r_06_prevencion_sp_trabajo,r_07_programa_ayuda_sp,r_08_c_consumo_marihuana_trabajo,r_08_d_consumo_cocaina_trabajo,r_09_rendimiento_disminuido_trabajo
0,3,1,54,3,2,Boyaca,TUNJA,2,1,3.0,...,2,NaN,0,NaN,2,2,2,9,9,2
1,6,2,46,2,2,Sucre,COROZAL,1,1,1.0,...,2,NaN,0,NaN,2,2,2,9,9,2
2,7,1,33,2,2,Sucre,COROZAL,1,1,1.0,...,2,NaN,0,NaN,1,1,1,9,9,9
3,8,2,33,2,2,Cordoba,MONTERÍA,2,1,3.0,...,2,NaN,0,NaN,9,2,9,9,9,2
4,11,2,27,2,2,Cordoba,MONTERÍA,2,1,3.0,...,2,NaN,0,NaN,1,2,9,9,9,2


Es conveniente renombrar las columnas y darles un nombre descriptivo

In [7]:
df_consumo_trabajo.dtypes

directorio                                        int64
per_sexo                                          int64
per_edad                                          int64
vive_padre_hogar                                  int64
vive_madre_hogar                                  int64
departamento                                     object
municipio                                        object
tipo_vivienda                                     int64
servicio_energia                                  int64
estrato                                         float64
residentes_hogar                                  int64
d_01_aporta_dinero_hogar                          int64
d_03_actividad_economica_empresa_negocio          int64
d_04_cargo                                        int64
d_08_estado_salud                                 int64
d_09_deprimido                                    int64
d_10_poco_interes                                 int64
d_11_h_riesgo_fumar_marihuana_frecuentemente    

In [8]:
df_consumo_trabajo = df_consumo_trabajo.rename(columns={
    "d_11_h_riesgo_fumar_marihuana_frecuentemente":"d_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente",
    "d_11_k_riesgo_cocaina_frecuentemente":"d_11_k_conocimiento_riesgo_cocaina_frecuentemente",
    "d_11_n_riesgo_fumar_basuco_frecuentemente":"d_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente",
    "d_12_b_consumo_sp_barrio":"d_12_b_presenta_problemas_consumo_sp_barrio",
    "d_12_c_expendio_sp_barrio":"d_12_c_presenta_problemas_expendio_sp_barrio",
    "r_04_consecuencia_sp":"r_04_consecuencia_sp_r_03",
    "r_07_programa_ayuda_sp":"r_07_programa_ayuda_sp_trabajo",
    "r_08_c_consumo_marihuana_trabajo":"r_08_c_consumo_marihuana_en_trabajo",
    "r_08_d_consumo_cocaina_trabajo":"r_08_d_consumo_cocaina_en_trabajo",
})

In [10]:
df_consumo_trabajo.dtypes

directorio                                                     int64
per_sexo                                                       int64
per_edad                                                       int64
vive_padre_hogar                                               int64
vive_madre_hogar                                               int64
departamento                                                  object
municipio                                                     object
tipo_vivienda                                                  int64
servicio_energia                                               int64
estrato                                                      float64
residentes_hogar                                               int64
d_01_aporta_dinero_hogar                                       int64
d_03_actividad_economica_empresa_negocio                       int64
d_04_cargo                                                     int64
d_08_estado_salud                 

**¿Cual es el objetivo de este análisis?**

Un aspecto importante que me gusta siempre tener claro al empezar un EDA es saber el objetivo del mismo. Generalmente hay una lista de preguntas concretas que responder, o al menos las personas a cargo de recolectar el dataset lo crearon con un objetivo en mente.

En este caso, uno de los objetivos por los cuales la EPA crea este dataset es para identificar factores de riesgo relacionadas al consumo de sustancias por parte de las personas, 


en este caso las variables que miden el consumo de marihuana, cocaina y bazuco

### Descripcion de entidad

- Departamento-municipio
- tipo_vivienda-servicio_energia
- tipo_vivienda-estrato
- tipo_vivienda-residentes_hogar

### Despues de cada paso es importante guardar el dataset. Asi no modificamos el original

In [9]:
df_consumo_trabajo.to_csv("Data/df_consumo_trabajo.1.procesado_inicial.csv", index=False)